# Experiment Viewer

We can visualize hyperparameter sweep result directly on Colab.



```
# This is formatted as code
```

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/brax/blob/main/experimental/braxlines/notebooks/vgcrl.ipynb)

In [ ]:
# @title Colab setup and imports
try:
  import brax
except ImportError:
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

output_path = '' #@param{'type': 'string'}

from brax.io import file
from brax.experimental.braxlines.common import logger_utils

pattern = f'{output_path}/**/training_curves.csv'
csv_files = file.Glob(pattern)
print(f'Found {len(csv_files)} files matching {pattern}')


In [ ]:
# @title Load data
data = {}
for i, csv_file in enumerate(csv_files):
  key = os.path.basename(os.path.dirname(csv_file))
  print(f'[{i+1}/{len(csv_files)}]')
  data[key] = logger_utils.parse_csv(csv_file, verbose=True)

In [ ]:
# @title Plot data
import matplotlib.pyplot as plt
import numpy as np
import re

def spec(N):
  t = np.linspace(-510, 510, N)
  return np.clip(np.stack([-t, 510 - np.abs(t), t], axis=1), 0, 255) / 255.

xlabel = 'num_steps' # @param['num_steps']
ylabel = 'eval/episode_reward' # @param['eval/episode_reward', 'losses/disc_loss']
filter_regex = '' # @param{'type': 'string'}
filter_regex = f'.*{filter_regex}.*'

fig, ax = plt.subplots(figsize=(7,6))
colors = spec(len(data))
for (k, v), c in zip(sorted(data.items()), colors):
  if not re.match(filter_regex, k):
    continue
  if ylabel in v:
    ax.plot(v[xlabel], v[ylabel], label=k, c=c)
ax.set(xlabel=xlabel, ylabel=ylabel)
ax.legend()
fig.tight_layout()
plt.show()